# Imports

In [179]:
# Imports
import numpy as np
import random
# Pythonic types
from typing import Callable
import itertools

# Introduction

## Problem 1: Generating Random Boolean Functions
- W.I.P Explain the theory in depth, talk abt defenitions of e.g. oracle and use diagrams
- reference work, notebook, websites + learning video https://www.youtube.com/watch?v=QcK0GK7DUh8&t=655s

### Problem Description
The Deutsch–Jozsa algorithm is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either constant (always returns False or always returns True) or balanced (returns True for exactly half of the possible input combinations). Write a Python function random_constant_balanced that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean arguments as inputs.

### 1.1 Helper function

In [180]:
def bin_args_to_int(*args: bool) -> int:
    """
    Takes an arbitrary number of Boolean (truthy/falsy) arguments 
    and converts them to a single integer using bitwise operations.
    
    Parameters:
        *args (bool): A variable number of Boolean inputs.
        
    Returns:
        int: The integer representation of the binary arguments.
    """

    # Initialize the result as 0
    result = 0

    # Iterate over each argument
    for arg in args:
        # Shift the current result left by 1 bit to make room for the new bit
        # Use bitwise OR to append the boolean value (treated as 1 or 0)
        result = (result << 1) | arg

    # After processing all arguments, 'result' will contain the combined integer value
    return result

### 1.2 Main Function: The Oracle Generator
The `random_constant_balanced` function acts as an oracle factory. Since we have $n=4$ Boolean inputs, there are $2^4 = 16$ possible input combinations. 

The function randomly decides whether to generate a constant or balanced truth table:
* **Constant:** A list of 16 `True` values or 16 `False` values.
* **Balanced:** A list containing exactly 8 `True` and 8 `False` values, shuffled randomly.

It then returns an inner function (a closure) that takes exactly four Boolean arguments, calculates the index using our helper function, and returns the corresponding value from the hidden truth table.

In [181]:
# fclosure to create a black box function that is either constant or balanced, preventing user from knowing function type. 
def random_constant_balanced() -> Callable[[bool, bool, bool, bool], bool]:
    """
    Generates a random black-box function that is either strictly constant 
    or strictly balanced for 4 Boolean inputs.
    
    Returns:
        Callable: A function that takes exactly 4 boolean arguments and 
                  returns a single boolean value.
    """

    # Decide randomly if the function will be constant or balanced
    is_constant = random.choice([True, False])

    if is_constant:
        # Constant: 16 True values OR 16 False values
        # 16 values as there are 16 possible combinations of 4 Boolean inputs (2^4 = 16)
        constant_value = random.choice([True, False])
        truth_table = [constant_value] * 16
    else:
        # Balanced: exactly 8 True values and 8 False values
        truth_table = [True] * 8 + [False] * 8
        random.shuffle(truth_table)
        
    # Defnining the inner function that takes exactly 4 Boolean arguments
    def f(b1: bool, b2: bool, b3: bool, b4: bool) -> bool:

        # Return the corresponding Boolean from our truth table
        return truth_table[bin_args_to_int(b1, b2, b3, b4)]
        
    return f

### 1.3 Demonstration
We can generate a single random function and inspect its behavior. <br>
By passing all 16 possible combinations of 4 Boolean inputs (generated via `itertools.product`) into our black-box function, we can determine its type based purely on the outputs.

In [182]:
# Generating a random function
f = random_constant_balanced()

# Generating all 16 possible 4-bit Boolean input combinations
all_inputs = list(itertools.product([False, True], repeat=4))

# Evaluating the function for all combinations
outputs = [f(*inp) for inp in all_inputs]
true_count = sum(outputs)

# Determining the type based strictly on the outputs
func_type = "Constant" if true_count in (0, 16) else "Balanced"

# Displaying the results
print(f"Generated Function Type: {func_type}")
print(f"Total True outputs:  {true_count}")
print(f"Total False outputs: {16 - true_count}\n")

# Display the formatted truth table
print(" b1 b2 b3 b4 | Output")
print("-" * 22)

# Formatting each Boolean as 0/1 for clean visual readability
for inp, out in zip(all_inputs, outputs):
    bits = "  ".join(str(int(b)) for b in inp)

    # Display the input bits and the corresponding output
    print(f"  {bits} | {out}")

Generated Function Type: Constant
Total True outputs:  16
Total False outputs: 0

 b1 b2 b3 b4 | Output
----------------------
  0  0  0  0 | True
  0  0  0  1 | True
  0  0  1  0 | True
  0  0  1  1 | True
  0  1  0  0 | True
  0  1  0  1 | True
  0  1  1  0 | True
  0  1  1  1 | True
  1  0  0  0 | True
  1  0  0  1 | True
  1  0  1  0 | True
  1  0  1  1 | True
  1  1  0  0 | True
  1  1  0  1 | True
  1  1  1  0 | True
  1  1  1  1 | True


### 1.4 Statistically Verification
To statistically verify the correctness of our function generator we will generate 1,000 random functions and assert that <br>
Every function evaluates to exactly 0, 16, or 8 `True` values across all combinations.

In [183]:
def test_random_constant_balanced(num_trials: int = 1000):
    """
    Tests the generator by creating multiple functions and validating 
    their outputs against the rules of the Deutsch-Jozsa algorithm.
    """
    all_inputs = list(itertools.product([False, True], repeat=4))
    counts = {"Constant": 0, "Balanced": 0}

    for _ in range(num_trials):
        func = random_constant_balanced()
        
        # create a list of outputs for all 16 input combinations
        outputs = [func(*inp) for inp in all_inputs]
        
        # validate that the function is either constant or balanced
        true_count = sum(outputs)
        assert true_count in (0, 8, 16), f"Invalid True count: {true_count} (Expected 0, 8, or 16)."
        
        # Tally the results
        if true_count in (0, 16):
            counts["Constant"] += 1
        else:
            counts["Balanced"] += 1

    print(f"All {num_trials} trials passed successfully.")
    print(f"Total Constant functions generated: {counts['Constant']}")
    print(f"Total Balanced functions generated: {counts['Balanced']}\n")

# Execute the test
test_random_constant_balanced()

All 1000 trials passed successfully.
Total Constant functions generated: 528
Total Balanced functions generated: 472



## Problem 2: Classical Testing for Function Type

## Problem 3: Quantum Oracles

## Problem 4: Deutsch's Algorithm with Qiskit

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm